In [1]:
#adding pydecomp 
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "/home/lestandi/Documents/thesis_light/prog/python/python_decomposition_library/pydecomp/")
sys.path.insert(0, "/home/lestandi/Documents/thesis_light/prog/python/python_decomposition_library/pydecomp/core/")
sys.path.insert(0, "/home/lestandi/Documents/thesis_light/prog/python/python_decomposition_library/pydecomp/utils/")
sys.path.insert(0, "/home/lestandi/Documents/thesis_light/prog/python/python_decomposition_library/pydecomp/analysis/")
sys.path.insert(0, "/home/lestandi/Documents/thesis_light/prog/python/python_decomposition_library/pydecomp/")

In [2]:
from pydecomp.core.TSVD import TSVD
import numpy as np

In [3]:
    #test procedure for the standard numpy matrix SVD
    from time import time
    print("\n Testing SVD with random matrix\n")
    n,m=10000,500
    F=np.random.rand(n,m)
    # F=np.reshape(np.arange(n*m),(n,m))

    t=time()
    phi, sigma, A=TSVD(F,solver='EVD',rank=5)
    print('\n --------------SVD by EVD computing time {}',time()-t)
    # print("\nEVD\n----------- \nphi :\n {}\n sigma:\n {} \n A:\n {}".format(phi, sigma, A))
    r=sigma.size
    sigma=np.reshape(sigma,[r,1])
    F_approx = phi@(sigma*A.T)
    err=np.linalg.norm(F_approx-F)
    print("r={}".format(r))
    print("sigma[:5]=",sigma[0:5])
    print("\n Should be small : {}".format(err))    
    
    t=time()
    phi, sigma, A=TSVD(F,solver='PRIMME',rank=5,epsilon=1e-6)
    print('\n --------------PRIMME_SVDs computing time {}',time()-t)
    # print("\nEVD\n----------- \nphi :\n {}\n sigma:\n {} \n A:\n {}".format(phi, sigma, A))
    r=sigma.size
    sigma=np.reshape(sigma,[r,1])
    F_approx = phi@(sigma*A.T)
    err=np.linalg.norm(F_approx-F)
    print("r={}".format(r))
    print("sigma[:5]=",sigma[0:5])
    print("\n Should be small : {}".format(err))


 Testing SVD with random matrix


 --------------SVD by EVD computing time {} 0.10413932800292969
r=5
sigma[:5]= [[1118.26018321]
 [  35.26072746]
 [  35.14429884]
 [  35.0374459 ]
 [  35.00972335]]

 Should be small : 641.0968003206452
Selected PRIMME_SVDS solver. This solver is iterative and best               suited for sparse tall skinny matrices. High accuracy requirement               may lead to intractable CPU times.

 --------------PRIMME_SVDs computing time {} 0.7990212440490723
r=5
sigma[:5]= [[1118.26018321]
 [  35.26072745]
 [  35.14429883]
 [  35.03744575]
 [  35.00972313]]

 Should be small : 641.0968003422727


In [4]:
import scipy
import primme
# Sparse random rectangular matrix 10^5x100
#A = scipy.sparse.rand(100000, 100, density=1., random_state=10)
#print(A.size)
# Compute the three closest singular values to 6.0 with a tolerance of 1e-6
svecs_left, svals, svecs_right, stats = primme.svds(F, 5, which='LM',
                                                    return_stats=True)
A_svals = svals
print(svals)
print(stats["elapsedTime"], stats["numMatvecs"])

[1118.26018321   35.26072746   35.14429884   35.0374459    35.00972335]
1.7337520122528076 555


In [5]:
(sigma[:,0]-svals)/sigma[:,0]

array([ 8.13312246e-16, -2.43447755e-10, -3.49641108e-10, -4.39244237e-09,
       -6.31406643e-09])

In [6]:
np.set_printoptions(precision=3)
print("phi orthog test")
print(phi.T@phi)
print("svecs orthog test")
print(svecs_left.T@svecs_left)

print("v orthog test")
print(A.T@A)
print("svecs orthog test")
print(svecs_right@svecs_right.T)

phi orthog test
[[ 1.000e+00  4.729e-15 -6.196e-15 -1.191e-15 -1.524e-15]
 [ 4.729e-15  1.000e+00  1.419e-13  5.159e-14  4.261e-14]
 [-6.196e-15  1.419e-13  1.000e+00 -6.390e-14  5.758e-14]
 [-1.191e-15  5.159e-14 -6.390e-14  1.000e+00 -5.952e-14]
 [-1.524e-15  4.261e-14  5.758e-14 -5.952e-14  1.000e+00]]
svecs orthog test
[[ 1.000e+00  4.063e-15 -7.689e-15 -1.492e-15  9.828e-15]
 [ 4.063e-15  1.000e+00 -2.948e-13 -6.607e-14 -1.344e-15]
 [-7.689e-15 -2.948e-13  1.000e+00 -2.257e-13 -2.894e-14]
 [-1.492e-15 -6.607e-14 -2.257e-13  1.000e+00  2.666e-14]
 [ 9.828e-15 -1.344e-15 -2.894e-14  2.666e-14  1.000e+00]]
v orthog test
[[ 1.000e+00  1.015e-16 -1.509e-16  8.327e-17 -1.787e-16]
 [ 1.015e-16  1.000e+00  5.716e-16 -3.305e-16 -1.388e-17]
 [-1.509e-16  5.716e-16  1.000e+00  1.471e-15  4.666e-16]
 [ 8.327e-17 -3.305e-16  1.471e-15  1.000e+00 -7.199e-16]
 [-1.787e-16 -1.388e-17  4.666e-16 -7.199e-16  1.000e+00]]
svecs orthog test
[[ 1.000e+00  8.760e-17  2.411e-16  1.596e-16 -2.082e-17]
 [ 

In [7]:
import math
import numpy as np
q=2
shape=[65,64]
size=np.product(shape)
D=int(round(math.log( size, q )))
new_shape=[q] * D
tuple( [ q**(int(math.log(s-0.5,q))+1) for s in shape ] )

(128, 64)

In [8]:
print(new_shape)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [9]:
A=np.ones(new_shape)

In [10]:
B=np.ones(shape)

In [11]:
A.size==B.size

False

In [12]:
(math.log( size, q )).is_integer()

False

In [13]:
from pydecomp.core import QuanticsTT

In [14]:
QuanticsTT.run_test()

reshaping to [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1]
[1 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 1]
[1 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 1]
[1 2 4 5 5 5 5 5 5 5 5 5 5 5 5 5 4 2 1]
[1 2 4 6 6 6 6 6 6 6 6 6 6 6 6 6 4 2 1]
[1 2 4 7 7 7 7 7 7 7 7 7 7 7 7 7 4 2 1]
[1 2 4 8 8 8 8 8 8 8 8 8 8 8 8 8 4 2 1]
[1 2 4 8 9 9 9 9 9 9 9 9 9 9 9 8 4 2 1]
[ 1  2  4  8 10 10 10 10 10 10 10 10 10 10 10  8  4  2  1]
[ 1  2  4  8 15 15 15 15 15 15 15 15 15 15 15  8  4  2  1]
[ 1  2  4  8 16 20 20 20 20 20 20 20 20 20 16  8  4  2  1]
[ 1  2  4  8 16 25 25 25 25 25 25 25 25 25 16  8  4  2  1]
[ 1  2  4  8 16 30 30 30 30 30 30 30 30 30 16  8  4  2  1]
[ 1  2  4  8 16 32 35 35 35 35 35 35 35 32 16  8  4  2  1]
[ 1  2  4  8 16 32 40 40 40 40 40 40 40 32 16  8  4  2  1]
[ 1  2  4  8 16 32 50 50 50 50 50 50 50 32 16  8  4  2  1]
[ 1  2  4  8 16 32 60 60 60 60 60 60 60 32 16  8  4  2  1]
[ 1  2  4  8 16 32 64 70 70 70 70 70 64 3

NameError: name 'qA' is not defined

In [19]:
decomp_methods=["QTT_SVD","SVD_by_EVD"]
N=32
benchmark_2D(decomp_methods ,shape=[N,N], test_function=2, plot=True,
                plot_name='pydecomp/plots/2D_approx_benchmark.pdf',tol=1e-6)

NameError: name 'benchmark_2D' is not defined

In [22]:
!python pydcomp.analysis.2D_benchmark.py

python: can't open file 'pydcomp.analysis.2D_benchmark.py': [Errno 2] No such file or directory
